In [1]:
import pandas as pd


In [2]:
CSV_FILE_PATH = './data/sample_taxi_after_clean.csv'
df = pd.read_csv(CSV_FILE_PATH)
df.head()

,taxi_id,time,lon,lat,is_passenger,speed
0,22224,00:00:12,114.039169,22.565283,0,45
1,22224,00:00:27,114.040001,22.563417,0,69
2,22224,00:00:42,114.041298,22.561033,0,70
3,22224,00:00:57,114.042366,22.559084,0,69
4,22224,00:01:23,114.043114,22.556850,0,23


In [3]:
df.lon = df.lon.astype('double')
df.lat = df.lat.astype('double')

In [4]:
taxi_counts = dict(df.taxi_id.value_counts())

In [5]:
id_list = list(taxi_counts.keys())

In [6]:
import haversine as hs

# loc1 = (loc1_lng, loc1_la)
# loc2 = (loc2_lng, loc2_la)
# dis = hs.haversine(loc1, loc2)

In [7]:
test = df[df.taxi_id == 22224]
test[test.duplicated('time')== False]

,taxi_id,time,lon,lat,is_passenger,speed
0,22224,00:00:12,114.039169,22.565283,0,45
1,22224,00:00:27,114.040001,22.563417,0,69
2,22224,00:00:42,114.041298,22.561033,0,70
3,22224,00:00:57,114.042366,22.559084,0,69
4,22224,00:01:23,114.043114,22.556850,0,23
...,...,...,...,...,...,...
4290,22224,23:52:42,114.431953,22.784201,1,92
4291,22224,23:52:57,114.434868,22.785467,1,83
4292,22224,23:53:12,114.437614,22.787018,1,83
4293,22224,23:53:27,114.440018,22.788668,1,80


In [8]:
OD_list = []
OD_path = []
for id in id_list:
    id_df = df[df.taxi_id == id]
    id_df = id_df.sort_values(by='time')
#     id_df = id_df[id_df.duplicated('time')== False]
    passenger_check = False

#     if (id_df.iloc[0]['is_passenger'] == 0):
#         passenger_check = False
#     else:
#         passenger_check = True
    distance = 0
    last_point = [0, 0, 0]
#     print(id_df.shape)
    for index, row in id_df.iterrows():
        if (len(row)==0):
            print(row)
            continue
        if (passenger_check and row['is_passenger'] == 0):
            passenger_check = False
            loc1 = (last_point[0], last_point[1])
            if (last_point == [0, 0, 0]):
                loc1 = (OD_path[0][1], OD_path[0][2])
            loc2 = (row['lon'], row['lat'])
            distance += hs.haversine(loc1, loc2)
            if (distance > 100):
                print(distance)
            
            OD_path.append([row['time'], row['lon'], row['lat']])
            OD_path.append(distance)
            OD_list.append(OD_path.copy())
            OD_path = []
            distance = 0
            last_point = [0, 0, 0]
        elif ((not passenger_check) and row['is_passenger'] == 1):
            passenger_check = True
            OD_path = []
            OD_path.append([row['time'], row['lon'], row['lat']])
#             print(OD_path)
            last_point = [0, 0, 0]
        elif passenger_check and row['is_passenger'] == 1 and last_point != [0, 0, 0]:
            loc1 = (last_point[0], last_point[1])
            loc2 = (row['lon'], row['lat'])
            distance += hs.haversine(loc1, loc2)
            if (distance > 100):
                print(distance)
            last_point[0] = row['lon']
            last_point[1] = row['lat']
            last_point[2] = row['time']

In [9]:
OD_list[0]

[['01:22:23', 113.842796, 22.613617],
 ['01:23:17', 113.845634, 22.61175],
 0.3265402530312467]

In [10]:
r_df1 = pd.DataFrame(
    columns=['from_time', 'lng1', 'lat1', 'to_time', 'lng2', 'lat2', 'distance'])
for i in OD_list:
    slide_series = pd.Series(
        {'from_time': i[0][0],
         'lng1': i[0][1],
         'lat1': i[0][2],
         'to_time': i[1][0],
         'lng2': i[1][1],
         'lat2': i[1][2],
         'distance': i[2]})
    r_df1 = r_df1.append(slide_series, ignore_index=True)
r_df1.to_csv("./data/OD_distance_data.csv", index=False, sep=',')


In [11]:
r_df1.describe()

,lng1,lat1,lng2,lat2,distance
count,107462.000000,107462.000000,107462.000000,107462.000000,107462.000000
mean,114.044177,22.573733,114.044207,22.575451,3.853796
std,0.096704,0.054546,0.097925,0.056150,5.108982
min,113.766800,22.357384,113.766785,22.325800,0.000000
25%,113.982216,22.538067,113.979118,22.538050,1.000147
50%,114.055481,22.552782,114.055496,22.554867,2.076694
75%,114.112644,22.602899,114.112984,22.605198,4.517206
max,114.480751,23.180433,114.602936,23.091999,73.397396
